In [ ]:
import time
import pandas as pd
import geopandas as gpd
import dataset
from keplergl import KeplerGl
from shapely.geometry import LineString, Point, MultiPoint
from simplification.cutil import (
    simplify_coords,
    simplify_coords_idx,
)

In [ ]:
%%time

db = dataset.connect('sqlite:///stravaweb/strava_activities.sqlite')
BASE_POINT = Point(44.01016, 56.31832)
SIMPLIFY_THRESHOLD = 0.0001

df = pd.DataFrame(x for x in db['activities'].all(_limit=None) if x['latlng'])


def gen_features(df, simplify_threshold=0.001):
    for latlng, times, base_datetime in zip(df.latlng, df.time, df.datetime):
        base_time = pd.to_datetime(base_datetime).timestamp()
        fixed = [(x[1], x[0]) for x in latlng]
        indexes = simplify_coords_idx(fixed, simplify_threshold)
        coords = [[fixed[i][0], fixed[i][1], 0, int(base_time + times[i])] for i in indexes]
#         print(len(indexes), len(times))
        yield {
            "type": "FeatureCollection",
            "features": [
              {
                "type": "Feature",
                "geometry": {
                  "type": "LineString",
                  "coordinates": coords,
                }
              }
            ]
          }

df['track'] = list(gen_features(df, SIMPLIFY_THRESHOLD))
df = df.drop(columns=['velocity_smooth', 'id', 'latlng', 'time'])

print(df.shape[0])
df.dtypes
# import json
# print(json.dumps(df.track[0]))

In [ ]:
import os
import json

config = None
if os.path.exists('map_config.json'):
    with open('map_config.json') as fp:
        config = json.load(fp)
        
my_map = KeplerGl(height=800, config=config)
my_map.add_data(df, 'tracks')
my_map

In [ ]:
my_map.save_to_html(file_name='tracks_map.html')

In [ ]:
def save_config(m):
    import json
    with open('map_config.json', 'w') as fp:
        json.dump(m.config, fp)
#save_config(my_map)